Input:
- True Observations
- Number Training Steps

Output:
- New Model Parameter

Parts of Optimization:
- Initialization of Model Parameter
- tbd

# Initialization

## Import Libraries and Mount Drive

In [1]:
import numpy as np
import pandas as pd
import torch

In [11]:
dtype = torch.float32
device = torch.device("cuda:0")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/GitHub/master-thesis

/content/drive/MyDrive/GitHub/master-thesis


In [6]:
import model.modeloptimization
from model.modelinitialisierung import initialize_observations

## Optimization Settings

In [8]:
start_time='2020-02-26'
end_time='2022-01-31'
observations=['Confirmed_cases']

## Initialize Observations

In [10]:
df_obs_initial = pd.read_csv('data/observations_03022022.csv' , sep=';', thousands='.', decimal=',', encoding='utf-8', parse_dates=['Date'])
df_obs = initialize_observations(df_obs_initial, start=start_time, end=end_time, observations=observations)
df_obs.head(3)

,Date,Confirmed_cases
0,2020-02-26,1
1,2020-02-27,1
2,2020-02-28,1


In [30]:
# Add Plot Observations

## Initialize Model Parameter

### Initialize Helper Parameter

In [ ]:
cero = torch.tensor(0., requires_grad=False, device=device, dtype=dtype)

### Initialize to be optimized Model Parameter

In [ ]:
R0_prime = torch.tensor(bij_transform_inv(torch.tensor(3.6), 2, 5), requires_grad=True, device=device, dtype=dtype)
phi_prime = torch.tensor(bij_transform_inv(torch.tensor(25), 0, 50), requires_grad=True, device=device, dtype=dtype)
sigma_prime = torch.tensor(bij_transform_inv(torch.tensor(0.1), 0.00001, 0.5), requires_grad=True, device=device, dtype=dtype)
alpha_prime = torch.tensor(bij_transform_inv(torch.tensor(0.01), 0.001, 0.05), requires_grad=True, device=device, dtype=dtype)

sigma = bij_transform(sigma_prime, 0.01, 0.5)

In [ ]:
epsilon_t = torch.zeros(num_observations, device=device)
epsilon_t[0] = torch.distributions.Normal(cero, sigma.detach()).rsample()
for t in range(1, num_observations):
    epsilon_t[t] = torch.distributions.Normal(epsilon_t[t - 1].detach(), sigma.detach()).rsample()
epsilon_t.requires_grad_(True)

# Optimization

## Loss Functions - To Do (adapt to new setting - maybe put in extra .py file)

In [ ]:
def calc_random_walk_loss(epsilon_t, sigma, training_days):
    loc = epsilon_t[:training_days-1]
    scale = sigma * torch.ones(training_days - 1, device=device)
    mvn = distributions.multivariate_normal.MultivariateNormal(loc, scale_tril=torch.diag(scale))
    ll = mvn.log_prob(epsilon_t[1:training_days])
    return -ll

In [ ]:
def calc_mse(expected, observed, training_days):
    diff = expected - observed[:training_days]
    square = diff.square()
    msr = square.mean()
    return msr

In [ ]:
dist_phi = distributions.normal.Normal(loc=torch.tensor([25], device=device), scale=torch.tensor([10], device=device))
dist_R0 = distributions.normal.Normal(loc=torch.tensor([3.6], device=device), scale=torch.tensor([0.8], device=device))
dist_alpha = distributions.normal.Normal(loc=torch.tensor([0.01], device=device), scale=torch.tensor([0.01], device=device))
dist_sigma = distributions.normal.Normal(loc=torch.tensor([0.05], device=device), scale=torch.tensor([0.03], device=device))

def calc_prior_loss(R0, sigma, phi, alpha):
    # log likelihood wrt. our prior ("regularisation")
    # ll stands for log-likelihood

    ll = torch.tensor(0.0, device=device)

    ll += dist_R0.log_prob(R0)[0] # neg

    ll += dist_sigma.log_prob(sigma)[0]

    ll += dist_phi.log_prob(phi)[0]

    ll += dist_alpha.log_prob(alpha)[0]

    return -ll

## Training

# Evaluation